# SP500 Dataset

In [6]:
import sqlite3
import pandas as pd
import numpy as np

import os
os.chdir('/home/ec2-user/SageMaker/')
os.getcwd()

'/home/ec2-user/SageMaker'

In [7]:
dbSP500 = sqlite3.connect('resources/phase3/sp500/sp500.sqlite')

# How large is this dataset?

In [10]:
queryTables = """
select name from sqlite_master where type = 'table'
"""

dfTables = pd.read_sql_query(queryTables, dbSP500)
print(dfTables)



    name
0  sp500


In [7]:
queryRows = """
select count(*) from sp500
"""

dfRows = pd.read_sql_query(queryRows, dbSP500)
print(dfRows)

   count(*)
0     17378


In [9]:
queryData = """
select * from sp500
"""

dfSP500 = pd.read_sql_query(queryData, dbSP500)
dfSP500.head(6)

,id,Date,Open,High,Low,Close,Adj Close,Volume
0,1,1950-01-03,16.66,16.66,16.66,16.66,16.66,1260000
1,2,1950-01-04,16.85,16.85,16.85,16.85,16.85,1890000
2,3,1950-01-05,16.93,16.93,16.93,16.93,16.93,2550000
3,4,1950-01-06,16.98,16.98,16.98,16.98,16.98,2010000
4,5,1950-01-09,17.08,17.08,17.08,17.08,17.08,2520000
5,6,1950-01-10,17.03,17.03,17.03,17.03,17.03,2160000


In [14]:
i = 0

for col in dfSP500:
    print(col)
    i += 1
    
print("\nFound %s columns" % i)

id
Date
Open
High
Low
Close
Adj Close
Volume

Found 8 columns


# What is the highest S&P500 price recorded in the dataset?

In [15]:
queryHighPrice = """
select max(high) from sp500
"""

resultHighPriceSQL = pd.read_sql_query(queryHighPrice, dbSP500)
print("SQL Result: \n\n %s" % resultHighPriceSQL)
print("\n\n\n")

resultHighPricePandas = dfSP500['High'].max()
print("Pandas result: \n\n %s" % resultHighPricePandas)

SQL Result: 

   max(high)
0    999.61




Pandas result: 

 999.61


# How much was the highest price and when was it?

In [18]:
queryHighPriceWhen = """
select * from sp500
where high = (select max(high) from sp500)
"""

resultHighPriceWhenSQL = pd.read_sql_query(queryHighPriceWhen, dbSP500)
print("SQL Result: \n\n %s" % resultHighPriceWhenSQL)
print("\n\n\n")

resultHighPriceWhenPandas = dfSP500[dfSP500['High']==dfSP500['High'].max()]
print("Pandas Result: \n\n %s" % resultHighPriceWhenPandas)

SQL Result: 

       id        Date    Open    High     Low   Close Adj Close      Volume
0  15004  2009-08-19  986.88  999.61  980.62  996.46    996.46  4257000000




Pandas Result: 

           id        Date    Open    High     Low   Close Adj Close      Volume
15003  15004  2009-08-19  986.88  999.61  980.62  996.46    996.46  4257000000


# What's the max, average, and min close price for latest 20 years in the S&P500 dataset?


In [21]:
queryYearlySummaryQuery = """
select
    substr(date, 1,4) as year,
    max(close) as max_close,
    avg(close) as avg_close,
    min(close) as min_close
from sp500
group by substr(date, 1,4)
order by substr(date, 1,4) desc
limit 20
"""

resultYearlySummarySQL = pd.read_sql_query(queryYearlySummaryQuery, dbSP500)
print("SQL Result: \n\n %s" % resultYearlySummarySQL)

SQL Result: 

     year max_close    avg_close min_close
0   2019   2670.71  2593.305294   2447.89
1   2018   2930.75  2746.214422    2351.1
2   2017   2690.16  2449.076375   2257.83
3   2016   2271.72  2094.651270   1829.08
4   2015   2130.82  2061.067738   1867.61
5   2014   2090.57  1931.376111   1741.89
6   2013   1848.36  1643.798968   1457.15
7   2012   1465.77  1379.354160   1277.06
8   2011   1363.61  1267.638810   1099.23
9   2010   1259.78  1139.965516   1022.58
10  2009    998.04   948.046389   1002.63
11  2008    998.01  1220.042055   1003.35
12  2007   1565.15  1477.184343   1374.12
13  2006   1427.09  1310.461235   1223.69
14  2005   1272.74  1207.229444    1137.5
15  2004   1213.55  1130.649444   1063.23
16  2003    999.74   965.227540    1000.3
17  2002    992.72   993.934802   1006.29
18  2001    984.54  1194.178992   1003.45
19  2000   1527.46  1427.221071   1264.74


In [22]:
dfSP500Work = dfSP500

dfSP500Work['Year'] = dfSP500Work['Date'].str.slice(stop=4)
dfSP500Work['Close'] = pd.to_numeric(dfSP500Work['Close'])

dfSP500Work.groupby(['Year']).agg({'Close': ['max', 'mean', 'min']}).sort_values(by = 'Year', ascending = False).head(20)

Close                      
          max         mean      min
Year                               
2019  2670.71  2593.305294  2447.89
2018  2930.75  2746.214422  2351.10
2017  2690.16  2449.076375  2257.83
2016  2271.72  2094.651270  1829.08
2015  2130.82  2061.067738  1867.61
2014  2090.57  1931.376111  1741.89
2013  1848.36  1643.798968  1457.15
2012  1465.77  1379.354160  1277.06
2011  1363.61  1267.638810  1099.23
2010  1259.78  1139.965516  1022.58
2009  1127.78   948.046389   676.53
2008  1447.16  1220.042055   752.44
2007  1565.15  1477.184343  1374.12
2006  1427.09  1310.461235  1223.69
2005  1272.74  1207.229444  1137.50
2004  1213.55  1130.649444  1063.23
2003  1111.92   965.227540   800.73
2002  1172.51   993.934802   776.76
2001  1373.73  1194.178992   965.80
2000  1527.46  1427.221071  1264.74

# My Question Examples:

# What was the lowest price and when did it occur most recently?

In [68]:
queryLowPriceWhen = """
select * from sp500
where low = (select min(low) from sp500)
order by date desc
limit 1
"""

resultLowPriceWhenSQL = pd.read_sql_query(queryLowPriceWhen, dbSP500)
print("Low Price: \n\n %s" % resultLowPriceWhenSQL)

Low Price: 

      id        Date    Open    High     Low  Close Adj Close    Volume
0  7328  1979-03-20  101.06  101.34  100.01  100.5     100.5  27180000


# When did the opening price equal the closing price in the 5 most recent circumstances?

In [64]:
queryEqualOpenAndClose = """
select * from sp500
where open = close
Order by date desc
limit 5
"""

resultEqualOpenAndCloseSQL = pd.read_sql_query(queryEqualOpenAndClose, dbSP500)
print("Opening and Closing Prices Equalled: \n\n %s" % resultEqualOpenAndCloseSQL)

Opening and Closing Prices Equalled: 

       id        Date     Open     High      Low    Close Adj Close      Volume
0  16927  2017-04-10  2357.16  2366.37   2351.5  2357.16   2357.16  2785410000
1  15292  2010-10-11  1165.32  1168.68  1162.02  1165.32   1165.32  2505900000
2  14207  2006-06-20  1240.12  1249.01  1238.87  1240.12   1240.12  2232950000
3  11844  1997-01-28   765.02   776.32   761.75   765.02    765.02   541580000
4  11327  1995-01-12   461.64   461.93   460.63   461.64    461.64   313040000


# How many days did the open price exceed the closing price ?

In [152]:
queryOpenPriceOverClosing = """
select count(*) from sp500
where open > close 
"""

resultOpenPriceOverClosingSQL = pd.read_sql_query(queryOpenPriceOverClosing, dbSP500)
print("Open Price exceeded Closing Price: \n\n %s" % resultOpenPriceOverClosingSQL)

Open Price exceeded Closing Price: 

    count(*)
0      6729


# What days were the closing price between 100 and 500?

In [198]:
queryClosingPriceLimit = """
select date, sum([close]) as closing_limit from sp500
group by date
having sum([close]) >= 100 and sum([close]) <= 500
order by date

"""

resultClosingPriceLimitSQL = pd.read_sql_query(queryClosingPriceLimit, dbSP500)
print("Closing price between 100 and 500: \n\n %s" % resultClosingPriceLimitSQL)

Closing price between 100 and 500: 

             Date  closing_limit
0     1968-06-04         100.38
1     1968-06-06         100.65
2     1968-06-07         101.27
3     1968-06-10         101.41
4     1968-06-11         101.66
5     1968-06-13         101.25
6     1968-06-14         101.13
7     1968-06-17         100.13
8     1968-06-20         101.51
9     1968-06-21         100.66
10    1968-06-24         100.39
11    1968-06-25         100.08
12    1968-07-03         100.91
13    1968-07-08         101.94
14    1968-07-09         102.23
15    1968-07-11         102.39
16    1968-07-12         102.34
17    1968-07-15         102.26
18    1968-07-16         101.70
19    1968-07-18         101.44
20    1968-07-19         100.46
21    1968-09-04         100.02
22    1968-09-05         100.74
23    1968-09-06         101.20
24    1968-09-09         101.23
25    1968-09-10         100.73
26    1968-09-12         100.52
27    1968-09-13         100.86
28    1968-09-16         101.24
29

# How many times does the open price exceed 1,000?

In [44]:
queryOPE = """
select date, sum([open]) as open_exceeding_one_thousand from sp500
group by date
having sum([open]) >= 1000
order by date

"""

resultOPESQL = pd.read_sql_query(queryOPE, dbSP500)
print("Closing price over 1000: \n\n %s" % resultOPESQL)

Closing price over 1000: 

             Date  open_exceeding_one_thousand
0     1998-02-03                      1001.27
1     1998-02-04                      1006.00
2     1998-02-05                      1006.90
3     1998-02-06                      1003.54
4     1998-02-09                      1012.46
5     1998-02-10                      1010.74
6     1998-02-11                      1019.01
7     1998-02-12                      1020.01
8     1998-02-13                      1024.14
9     1998-02-17                      1020.09
10    1998-02-18                      1022.76
11    1998-02-19                      1032.08
12    1998-02-20                      1028.28
13    1998-02-23                      1034.21
14    1998-02-24                      1038.14
15    1998-02-25                      1030.56
16    1998-02-26                      1042.90
17    1998-02-27                      1048.67
18    1998-03-02                      1049.34
19    1998-03-03                      1047.70
20    

# Which day had the highest increase from open to close?

In [43]:
queryIncrease = """
select date, max(open-close) as highest_increase from sp500
order by highest_increase 
limit 1
"""

resultIncreaseSQL = pd.read_sql_query(queryIncrease, dbSP500)
print("Date of highest increase from open to close: \n\n %s" % resultIncreaseSQL)

Date of highest increase from open to close: 

          Date  highest_increase
0  2018-02-08            104.01
